In [24]:
import sys
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data_local_computer

import torch
from collections import defaultdict

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import copy                    # clone tensor


In [46]:
lat_lon_resolution = [20,20]
day = 1
mm_cond_number = 20

years = ['2024']
month_range =[7,8]
idx_for_datamap= [ 8*(day-1),8*day]

instance = load_data_local_computer()
map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap=[0,8])


MLE using full likelihood

MLE using vecchia likelihood

In [13]:
instance = kernels.likelihood_function(smooth=0.5, input_map=analysis_data_map, aggregated_data=aggregated_data,nns_map=nns_map, mm_cond_number=mm_cond_number)

# Define your initial parameters
params = [25.96, 2.12, 2.225, 0.00167, -0.07942, 0.1025, 2.8709]
params = torch.tensor(params, requires_grad=True)
torch_smooth = torch.tensor(0.5, dtype=torch.float32)


for i in range(10,15):
    mm_cond_number = i 
    print("mm_cond_number: ", mm_cond_number)
    instance = kernels.likelihood_function(smooth=torch_smooth , input_map=analysis_data_map,aggregated_data=aggregated_data, nns_map=nns_map, mm_cond_number=mm_cond_number)

    out0 = instance.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_anisotropy_v05)
    print(out0)
    out0 = instance.vecchia_like_local_computer(params, instance.matern_cov_anisotropy_v05)
    print(out0)

    out0 = instance.vecchia_like_amarel(params, instance.matern_cov_anisotropy_v05)
    print(out0)


mm_cond_number:  10
tensor(2555.2388, grad_fn=<MulBackward0>)
tensor(2680.5918, grad_fn=<AddBackward0>)
tensor(2569.0044, grad_fn=<AddBackward0>)
mm_cond_number:  11
tensor(2555.2388, grad_fn=<MulBackward0>)
tensor(2680.8625, grad_fn=<AddBackward0>)
tensor(2569.3394, grad_fn=<AddBackward0>)
mm_cond_number:  12
tensor(2555.2388, grad_fn=<MulBackward0>)
tensor(2681.8325, grad_fn=<AddBackward0>)
tensor(2570.2656, grad_fn=<AddBackward0>)
mm_cond_number:  13
tensor(2555.2388, grad_fn=<MulBackward0>)
tensor(2681.7373, grad_fn=<AddBackward0>)
tensor(2570.2688, grad_fn=<AddBackward0>)
mm_cond_number:  14
tensor(2555.2388, grad_fn=<MulBackward0>)
tensor(2680.3740, grad_fn=<AddBackward0>)
tensor(2568.9595, grad_fn=<AddBackward0>)


Optimization vecchia

In [15]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = kernels.model_fitting(
    smooth=0.5,
    input_map=analysis_data_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number
)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer, scheduler = instance.optimizer_fun( params, lr=0.01, betas=(0.9, 0.99), eps=1e-8, step_size=10, gamma=0.9)  
instance.run_vecc_local(params, optimizer, scheduler,epochs=3000)

Epoch 1, Gradients: [ -1.7552054   -0.5213964   -0.40610468  11.0489435    1.7328523
 337.61792      5.033931  ]
 Loss: 2715.16455078125, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 101, Gradients: [-3.455997   -3.6594698  -1.3484838   0.5684591   0.05922849  2.5288239
 -5.5387034 ]
 Loss: 2673.765869140625, Parameters: [25.097012    2.4859247   2.4213777  -0.0418863  -0.05511027  0.09094857
  3.7057862 ]
Epoch 201, Gradients: [-3.2983828  -2.6702318  -0.8821263  -0.00876932 -0.00688924  1.4563928
 -4.626367  ]
 Loss: 2671.36083984375, Parameters: [25.319153    2.6351879   2.5463     -0.04287111 -0.0522955   0.08291206
  3.9088213 ]
Epoch 301, Gradients: [-3.2516239e+00 -2.3191965e+00 -7.0335937e-01 -2.7814806e-03
 -4.4769645e-03  9.0806866e-01 -4.3453703e+00]
 Loss: 2670.604248046875, Parameters: [25.396547    2.6918888   2.5932794  -0.04274024 -0.05125534  0.08015724
  3.9798565 ]
Converged at epoch 389
Epoch 390, Gradients: [-3.2

[array([25.42218   ,  2.7127497 ,  2.6107965 , -0.04270839, -0.05089707,
         0.07918856,  4.0041604 ], dtype=float32),
 2670.36181640625]

Vecchia experiments here

In [28]:
idx_for_datamap = [0,8]

class matern_advec_beta_torch_vecchia:
    def __init__(self, analaysis_data_map: torch.Tensor, params: torch.Tensor, nns_map=nns_map, mm_cond_number=mm_cond_number):
        
        self.key_list = sorted(analysis_data_map)
        self.input_map = analysis_data_map

        self.mm_cond_number = mm_cond_number
        self.nns_map = nns_map 
        self.analaysis_data_map = analaysis_data_map
        self.smooth = 0.5  
        sample_df = analaysis_data_map[self.key_list[0]]

        self.size_per_hour = len(sample_df)

    def custom_distance_matrix(self, U, V):
        # Efficient distance computation with broadcasting
        spatial_diff = torch.norm(U[:, :2].unsqueeze(1) - V[:, :2].unsqueeze(0), dim=2)

        temporal_diff = torch.abs(U[:, 2].unsqueeze(1) - V[:, 2].unsqueeze(0))
        distance = (spatial_diff**2 + temporal_diff**2)  # move torch.sqrt to covariance function to track gradients of beta and avec
        return distance
    
    def precompute_coords_ani(self, params, y: torch.Tensor, x: torch.Tensor)-> torch.Tensor:
        sigmasq, range_lat, range_lon, advec_lat, advec_lon, beta, nugget = params

        if y is None or x is None:
            raise ValueError("Both y and x_df must be provided.")

        x1 = x[:, 0]
        y1 = x[:, 1]
        t1 = x[:, 3]

        x2 = y[:, 0]
        y2 = y[:, 1]
        t2 = y[:, 3]

        # spat_coord1 = torch.stack((self.x1 , self.y1 - advec * self.t1), dim=-1)
        spat_coord1 = torch.stack(( (x1 - advec_lat * t1)/range_lat, (y1 - advec_lon * t1)/range_lon ), dim=-1)
        spat_coord2 = torch.stack(( (x2 - advec_lat * t2)/range_lat, (y2 - advec_lon * t2)/range_lon ), dim=-1)

        U = torch.cat((spat_coord1, (beta * t1).reshape(-1, 1)), dim=1)
        V = torch.cat((spat_coord2, (beta * t2).reshape(-1, 1)), dim=1)

        distance = self.custom_distance_matrix(U,V)
        non_zero_indices = distance != 0
        return distance, non_zero_indices
    
    # anisotropic in three 
    def matern_cov_ani(self,params: torch.Tensor, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        sigmasq, range_lat, range_lon, advec_lat, advec_lon, beta, nugget = params
        

        distance, non_zero_indices = self.precompute_coords_ani(params, x,y)
        out = torch.zeros_like(distance)

        non_zero_indices = distance != 0
        if torch.any(non_zero_indices):
            out[non_zero_indices] = sigmasq * torch.exp(- torch.sqrt(distance[non_zero_indices]))
        out[~non_zero_indices] = sigmasq

        # Add a small jitter term to the diagonal for numerical stability
        out += torch.eye(out.shape[0]) * nugget

        return out
    
    def full_likelihood(self,params: torch.Tensor, input_np: torch.Tensor, y: torch.Tensor, covariance_function) -> torch.Tensor:
        input_arr = input_np[:, :4]
        y_arr = y

        # Compute the covariance matrix
        cov_matrix = covariance_function(params=params, y=input_arr, x=input_arr)
        
        # Compute the log determinant of the covariance matrix
        sign, log_det = torch.slogdet(cov_matrix)
        # if sign <= 0:
        #     raise ValueError("Covariance matrix is not positive definite")
        
        # Extract locations
        locs = input_arr[:, :2]

        # Compute beta
        tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
        tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
        beta = torch.linalg.solve(tmp1, tmp2)

        # Compute the mean
        mu = torch.matmul(locs, beta)
        y_mu = y_arr - mu

        # Compute the quadratic form
        quad_form = torch.matmul(y_mu, torch.linalg.solve(cov_matrix, y_mu))

        # Compute the negative log likelihood
        neg_log_lik = 0.5 * (log_det + quad_form)
     
        return  neg_log_lik 
    

    def vecchia_like_local_computer(self, params: torch.Tensor, covariance_function) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        
        for time_idx in range(len(self.analaysis_data_map)):
            current_np = self.analaysis_data_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            cur_heads = current_np[:21, :]
            neg_log_lik += self.full_likelihood(params, cur_heads, cur_heads[:, 2], covariance_function)

            for index in range(21, self.size_per_hour):

                
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors)
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 1:
                    cov_matrix = self.cov_map[index]['cov_matrix']
                    tmp_for_beta = self.cov_map[index]['tmp_for_beta']
                    cov_xx_inv = self.cov_map[index]['cov_xx_inv']
                    L_inv = self.cov_map[index]['L_inv']
                    cov_ygivenx = self.cov_map[index]['cov_ygivenx']
                    cond_mean_tmp = self.cov_map[index]['cond_mean_tmp']
                    log_det = self.cov_map[index]['log_det']
                    locs = self.cov_map[index]['locs']
                    
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                    if data_list:
                        conditioning_data = torch.vstack(data_list)
                    else:
                        conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                    np_arr = torch.vstack((current_row, conditioning_data))
                    y_and_neighbors = np_arr[:, 2]

                    cov_yx = cov_matrix[0, 1:]

                    tmp2 = torch.matmul(torch.matmul(L_inv, locs).T, torch.matmul(L_inv, y_and_neighbors))
                    beta = torch.linalg.solve(tmp_for_beta, tmp2)

                    mu = torch.matmul(locs, beta)
                    mu_current = mu[0]
                    mu_neighbors = mu[1:]
                    
                    # Mean and variance of y|x
                    cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                    alpha = current_y - cond_mean
                    quad_form = alpha**2 * (1 / cov_ygivenx)
                    neg_log_lik += 0.5 * (log_det + quad_form)

                    continue

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                L = torch.linalg.cholesky(cov_matrix)
                L11 = L[:1, :1]
                L12 = torch.zeros(L[:1, 1:].shape)
                L21 = L[1:, :1]
                L22 = L[1:, 1:]
                L11_inv = torch.linalg.inv(L11)
                L22_inv = torch.linalg.inv(L22)

                L_inv = torch.block_diag(L11_inv, L22_inv)
                L_inv[1:, :1] = -torch.matmul(L22_inv, L21) @ L11_inv

                cov_yx = cov_matrix[0, 1:]

                tmp1 = torch.matmul(L_inv, locs)
                tmp2 = torch.matmul(torch.matmul(L_inv, locs).T, torch.matmul(L_inv, y_and_neighbors))
                tmp_for_beta = torch.matmul(tmp1.T, tmp1)
                beta = torch.linalg.solve(tmp_for_beta, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)

             
                if time_idx == 1:
                    self.cov_map[index] = {
                        'tmp_for_beta': tmp_for_beta,
                        'cov_xx_inv': cov_xx_inv,
                        'cov_matrix': cov_matrix,
                        'L_inv': L_inv,
                        'cov_ygivenx': cov_ygivenx,
                        'cond_mean_tmp': cond_mean_tmp,
                        'log_det': log_det,
                        'locs': locs
                    }

        return neg_log_lik  
    


In [ ]:
class torch_vecchia_exp(matern_advec_beta_torch_vecchia):
    def __init__(self, analaysis_data_map: torch.Tensor, params: torch.Tensor, nns_map=nns_map, mm_cond_number=mm_cond_number):
        super().__init__(analaysis_data_map, params, nns_map, mm_cond_number)
        # Any additional initialization for dignosis class can go here

    def vecchia_local2(self, params: torch.Tensor, covariance_function) -> torch.Tensor:
        self.cov_map = defaultdict(list)
        neg_log_lik = 0.0
        
        for time_idx in range(len(self.analaysis_data_map)):
            current_np = self.analaysis_data_map[self.key_list[time_idx]]

            # Use below when working on local computer to avoid singular matrix
            cur_heads = current_np[:21, :]
            neg_log_lik += self.full_likelihood(params, cur_heads, cur_heads[:, 2], covariance_function)

            for index in range(21, self.size_per_hour):

                
                current_row = current_np[index].reshape(1, -1)
                current_y = current_row[0, 2]

                # Construct conditioning set
                mm_neighbors = self.nns_map[index]
                past = list(mm_neighbors)
                data_list = []

                if past:
                    data_list.append(current_np[past])

                if time_idx > 1:
                    cov_matrix = self.cov_map[index]['cov_matrix']
                    tmp_for_beta = self.cov_map[index]['tmp_for_beta']
                    cov_xx_inv = self.cov_map[index]['cov_xx_inv']
            
                    cov_ygivenx = self.cov_map[index]['cov_ygivenx']
                    cond_mean_tmp = self.cov_map[index]['cond_mean_tmp']
                    log_det = self.cov_map[index]['log_det']
                    locs = self.cov_map[index]['locs']
                    
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                    if data_list:
                        conditioning_data = torch.vstack(data_list)
                    else:
                        conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                    np_arr = torch.vstack((current_row, conditioning_data))
                    y_and_neighbors = np_arr[:, 2]

                    cov_yx = cov_matrix[0, 1:]

                    y_arr = y_and_neighbors
                    tmp1 = tmp_for_beta
                    tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                    beta = torch.linalg.solve(tmp1, tmp2)

                    mu = torch.matmul(locs, beta)
                    mu_current = mu[0]
                    mu_neighbors = mu[1:]
                    
                    # Mean and variance of y|x
                    cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                    alpha = current_y - cond_mean
                    quad_form = alpha**2 * (1 / cov_ygivenx)
                    neg_log_lik += 0.5 * (log_det + quad_form)

                    continue

                if time_idx > 0:
                    last_hour_np = self.input_map[self.key_list[time_idx - 1]]
                    past_conditioning_data = last_hour_np[past + [index], :]
                    data_list.append(past_conditioning_data)

                if data_list:
                    conditioning_data = torch.vstack(data_list)
                else:
                    conditioning_data = torch.empty((0, current_row.shape[1]), dtype=torch.float32)

                np_arr = torch.vstack((current_row, conditioning_data))
                y_and_neighbors = np_arr[:, 2]
                locs = np_arr[:, :2]

                cov_matrix = covariance_function(params=params, y= np_arr, x= np_arr)
                # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
                cov_yx = cov_matrix[0, 1:]
                        # Compute the log determinant of the covariance matrix
                sign, log_det = torch.slogdet(cov_matrix)
                # if sign <= 0:
                #     raise ValueError("Covariance matrix is not positive definite")
            
                y_arr = y_and_neighbors
                # Compute beta
                tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
                tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
                beta = torch.linalg.solve(tmp1, tmp2)

                mu = torch.matmul(locs, beta)
                mu_current = mu[0]
                mu_neighbors = mu[1:]

                # Mean and variance of y|x
                sigma = cov_matrix[0, 0]
                cov_xx = cov_matrix[1:, 1:]
                cov_xx_inv = torch.linalg.inv(cov_xx)

                cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
                cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
                cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
                
                alpha = current_y - cond_mean
                quad_form = alpha**2 * (1 / cov_ygivenx)
                log_det = torch.log(cov_ygivenx)
                neg_log_lik += 0.5 * (log_det + quad_form)
 
                if time_idx == 1:
                    self.cov_map[index] = {
                        'tmp_for_beta': tmp1,
                        'cov_xx_inv': cov_xx_inv,
                        'cov_matrix': cov_matrix,
               
                        'cov_ygivenx': cov_ygivenx,
                        'cond_mean_tmp': cond_mean_tmp,
                        'log_det': log_det,
                        'locs': locs
                    }
        return neg_log_lik
    


    def vecchia_local3(self, params: torch.Tensor, covariance_function) -> torch.Tensor:
        neg_log_lik = 0.0
        
        for idx in range(len(aggregated_data)):
            current_row = aggregated_data[idx,:4]
            conditioning_data = aggregated_data[:idx,:4]

            torch_arr = torch.vstack((current_row, conditioning_data))
            y_and_neighbors = torch_arr[:, 2]
            locs = torch_arr[:, :2]

            cov_matrix = covariance_function(params=params, y= torch_arr, x= torch_arr)
            # print(f'Condition number: {torch.linalg.cond(cov_matrix)}')
            cov_yx = cov_matrix[0, 1:]
                    # Compute the log determinant of the covariance matrix
            sign, log_det = torch.slogdet(cov_matrix)
            # if sign <= 0:
            #     raise ValueError("Covariance matrix is not positive definite")
        
            y_arr = y_and_neighbors
            # Compute beta
            tmp1 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, locs))
            tmp2 = torch.matmul(locs.T, torch.linalg.solve(cov_matrix, y_arr))
            beta = torch.linalg.solve(tmp1, tmp2)

            mu = torch.matmul(locs, beta)
            mu_current = mu[0]
            mu_neighbors = mu[1:]

            # Mean and variance of y|x
            sigma = cov_matrix[0, 0]
            cov_xx = cov_matrix[1:, 1:]
            cov_xx_inv = torch.linalg.inv(cov_xx)

            cov_ygivenx = sigma - torch.matmul(cov_yx, torch.matmul(cov_xx_inv, cov_yx))
            cond_mean_tmp = torch.matmul(cov_yx, cov_xx_inv)
            cond_mean = mu_current + torch.matmul(cond_mean_tmp, (y_and_neighbors[1:] - mu_neighbors))
            
            alpha = current_row[2] - cond_mean
            quad_form = alpha**2 * (1 / cov_ygivenx)
            log_det = torch.log(cov_ygivenx)
            neg_log_lik += 0.5 * (log_det + quad_form)
 
        return neg_log_lik



In [47]:

instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)
out0 = instance.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_ani)
print(out0)

out0 = instance.vecchia_local3(params, instance.matern_cov_ani)
print(out0)



tensor(671.3489, grad_fn=<MulBackward0>)
tensor(666.9693, grad_fn=<AddBackward0>)


In [49]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer = optim.Adam([params],lr=0.01, betas=(0.9, 0.8), eps=1e-8)
scheduler = StepLR(optimizer, step_size=20, gamma=0.9)

# Example function to compute out1
def compute_out1(params):
    # Compute the output using your function
    # nll = instance.full_likelihood(params, instance.matern_advec_beta_cov )
    nll = instance.vecchia_local3(params, instance.matern_cov_ani )
    return nll

# Training loop
prev_loss = float('inf')
tol = 1e-4  # Convergence tolerance
for epoch in range(500):  # Number of epochs
    optimizer.zero_grad()  # Zero the gradients 
    
    loss = compute_out1(params)
    loss.backward()  # Backpropagate the loss
    
    # Print gradients and parameters every 10th epoch
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    # print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    optimizer.step()  # Update the parameters
    scheduler.step()
    # Check for convergence
    if abs(prev_loss - loss.item()) < tol:
        print(f"Converged at epoch {epoch}")
        break
    
    prev_loss = loss.item()

print('Training complete.') 



Epoch 1, Gradients: [  0.6312605    0.71205693  -4.26063     -4.6273994   -6.812104
 126.09926      4.8840494 ]
 Loss: 673.9320068359375, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 101, Gradients: [-0.39419731 -0.10302929 -0.51027846 -0.8810717  -0.1707418  -1.7684574
 -0.2682714 ]
 Loss: 660.8019409179688, Parameters: [ 2.4863796e+01  2.1887534e+00  2.8069072e+00  1.0403260e-01
 -1.6217563e-02  9.7153194e-02  3.4280524e+00]
Epoch 201, Gradients: [-0.39027733 -0.00202319 -0.00925368 -0.15174448  0.14574727 -0.45103073
  0.00092871]
 Loss: 660.5121459960938, Parameters: [ 2.5346031e+01  2.3260264e+00  3.0814703e+00  1.0996702e-01
 -1.6138716e-02  8.6622350e-02  3.6904948e+00]
Converged at epoch 273
Training complete.


Experiment results

## 200 x 8

lr 0.001 without scheduler  same as lr, step_size, gamma  0.01 40 0.5  (9.8s)

 Loss: 2549.066650390625, full Parameters: [ 2.48777485e+01  2.05998826e+00  2.16013098e+00  2.20775465e-03
 -7.89414570e-02  1.05411254e-01  3.75236106e+00]

 lr 0.01  step size 40  betas 0.9 , 0.8 gamma 0.9  30 s

  Loss: 2547.1728515625, full Parameters: [ 2.7377291e+01  2.2077193e+00  2.3204505e+00  1.0307773e-03
 -8.0311157e-02  9.8579854e-02  3.6677265e+00]

 lr 0.01  step size 10 betas 0.9 , 0.8 gamma 0.9  30 s
  Loss: 2548.87841796875, full Parameters: [ 2.5092268e+01  2.0689390e+00  2.1694989e+00  2.0285936e-03
 -7.9028614e-02  1.0501490e-01  3.7373385e+00]
Training full likelihood complete.   11.8 sc

 lr 0.01  step size 20 betas 0.9 , 0.8 gamma 0.9  30 s
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]



lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9
 Loss: 2548.18603515625, full Parameters: [ 2.5938652e+01  2.1110108e+00  2.2155209e+00  1.5893303e-03
 -7.9482891e-02  1.0297947e-01  3.6958976e+00]
 21.6

lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]
 22.9 s

lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9
Loss: 2548.95361328125, full Parameters: [ 2.5118145e+01  1.9827319e+00  2.0768294e+00  1.0898338e-03
 -8.0070712e-02  1.1034889e-01  3.5647078e+00]


## 1250 x 8

1250* 8 55m using constant learning rate 0.0001 
Loss: 14068.798828125, full Parameters: [ 2.46198387e+01  1.61719894e+00  1.76454413e+00  8.55297223e-03
 -1.08275235e-01  1.28809512e-01  2.80795789e+00]

1250* 8 10m 32s
lr 0.01  step size 40 beta 0.9 0.8 gamma 0.9
  Loss: 14068.1953125, full Parameters: [ 2.5030930e+01  1.6107724e+00  1.7573007e+00  8.8407323e-03
 -1.0820019e-01  1.2936097e-01  2.7430327e+00]
Training full likelihood complete.

9m 33s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 14068.29296875, full Parameters: 
 [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03 -1.0737537e-01  1.2953614e-01 
  2.7420275e+00]
Training full likelihood complete.

#### high resolution data might benefits from larger step size high resolution data often provides 
#### more stable gradients, so larger step size less likely to cause significant fluctuations
14n 41.8s
lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9

FINAL STATE: Epoch 199, 
 Loss: 14068.8828125, full Parameters: 
 [ 2.4707581e+01  1.6489888e+00  1.7993137e+00  8.4043797e-03 -1.0836436e-01  1.2655504e-01  
 2.8416286e+00]

#### beta 0.9 0.99 might be too conservative for high resolution data
13m 44.8s
lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9

 Loss: 14068.318359375, full Parameters: [ 2.4938175e+01  1.6203119e+00  1.7678342e+00  8.6686825e-03
 -1.0813228e-01  1.2845081e-01  2.7731323e+00]


18m
lr 0.01  step size 40 beta 0.9 0.99 gamma 0.9

 Loss: 14067.970703125, full Parameters: [ 2.5205673e+01  1.6159834e+00  1.7630767e+00  8.7957922e-03
 -1.0802399e-01  1.2862283e-01  2.7390635e+00]

9m 52s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9

Loss: 14068.29296875, full Parameters: [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03
 -1.0737537e-01  1.2953614e-01  2.7420275e+00]
Training full likelihood complete.